In [7]:
!pip install dlib-bin==19.24.6
!pip install face_recognition_models
!pip install cmake
!pip install numpy
!pip install Pillow
!pip install click
!pip install opencv-python

  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
   ---------------------------------------- 0.0/36.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/36.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/36.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/36.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/36.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/36.9 MB 163.8 kB/s eta 0:03:46
   ---------------------------------------- 0.1/36.9 MB 280.5 kB/s eta 0:02:12
   ---------------------------------------- 0.1/36.9 MB 423.5 kB/s eta 0:01:27
   ---------------------------------------- 0.1/36.9 MB 448.2 kB/s eta 0:01:23
   ---------------------------------------- 0.2/36.9 MB 535.8 kB/s eta 0:01:09
   ---------------------------------------- 0.2/36.9 MB 528.4 kB/s eta 0:01:10
   ---------------------------------------- 0.3/36.9 MB 561.1 kB/s eta 0:01:06
   ----------------------------

In [14]:
!pip install face_recognition --no-deps

In [5]:
!pip install ultralytics

  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.0 MB 991.0 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.0 MB 1.1 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.0 MB 1.3 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.0 MB 1.3 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.0 MB 1.5 MB/s eta 0:00:01
   ---------------- ----------------------- 0.4/1.0 MB 1.5 MB/s eta 0:00:01
   ------------------- -------------------- 0.5/1.0 MB 1.6 MB/s eta 0:00:01
   ----------------------- ---------------- 0.6/1.0 MB 1.6 MB/s eta 0:00:01
   --------------------------- ------------ 0.7/1.0 MB 1.7 MB/s eta 0:00:01
   -------------------------------- ------- 0.8/1.0 MB 1.8 MB/s eta 0:00:01
   ----------------------------------- ---- 0.9/1.0 MB 1.7 MB/s eta 0:00:01
   ----------------

In [13]:
!pip install cvzone

  Using cached cvzone-1.6.1.tar.gz (25 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26309 sha256=ef7cc80c0b02ce8bef47abca8520d44796430da2474d6e2990d087521f214d44
  Stored in directory: c:\users\kiit0001\appdata\local\pip\cache\wheels\5d\21\e8\3147ae88d44e27f06e0175d337a7673c70fb957202cbbe2034
Successfully built cvzone


In [19]:
!pip install torch torchvision

In [73]:
minifasnet_full_code = '''
import torch
import torch.nn as nn
import torch.nn.functional as F

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class ConvBlock(nn.Module):
    def __init__(self, in_c, out_c, kernel_size, stride, padding):
        super().__init__()
        self.conv = nn.Conv2d(in_c, out_c, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_c)
        self.prelu = nn.PReLU()
    def forward(self, x):
        return self.prelu(self.bn(self.conv(x)))

class MiniFASNetV2(nn.Module):
    def __init__(self, num_classes=3, img_channel=3):
        super().__init__()
        self.conv1 = ConvBlock(img_channel, 32, 3, 2, 1)
        self.conv2_dw = ConvBlock(32, 32, 3, 1, 1)

        self.conv_23 = nn.Sequential(
            ConvBlock(32, 103, 1, 1, 0),
            ConvBlock(103, 103, 3, 2, 1),
            ConvBlock(103, 64, 1, 1, 0),
        )

        self.conv_3 = nn.Sequential(
            self._make_block(64, 13),
            self._make_block(64, 26),
            self._make_block(64, 13),
            self._make_block(64, 52),
        )

        self.conv_34 = nn.Sequential(
            ConvBlock(64, 231, 1, 1, 0),
            ConvBlock(231, 231, 3, 2, 1),
            ConvBlock(231, 128, 1, 1, 0),
        )

        self.conv_4 = nn.Sequential(
            self._make_block(128, 154),
            self._make_block(128, 52),
            self._make_block(128, 26),
            self._make_block(128, 52),
            self._make_block(128, 26),
            self._make_block(128, 26),
        )

        self.conv_45 = nn.Sequential(
            ConvBlock(128, 308, 1, 1, 0),
            ConvBlock(308, 308, 3, 2, 1),
            ConvBlock(308, 128, 1, 1, 0),
        )

        self.conv_5 = nn.Sequential(
            self._make_block(128, 26),
            self._make_block(128, 26),
        )

        self.conv_6_sep = ConvBlock(128, 512, 1, 1, 0)
        self.conv_6_dw = nn.Conv2d(512, 512, 7, groups=512, bias=False)

        self.flatten = Flatten()
        self.linear = nn.Linear(512, 128, bias=False)
        self.bn = nn.BatchNorm1d(128)
        self.dropout = nn.Dropout(p=0.2)
        self.prob = nn.Linear(128, num_classes)

    def _make_block(self, in_c, out_c):
        return nn.Sequential(
            ConvBlock(in_c, out_c, 1, 1, 0),
            ConvBlock(out_c, out_c, 3, 1, 1),
            ConvBlock(out_c, in_c, 1, 1, 0),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2_dw(x)
        x = self.conv_23(x)
        x = self.conv_3(x)
        x = self.conv_34(x)
        x = self.conv_4(x)
        x = self.conv_45(x)
        x = self.conv_5(x)
        x = self.conv_6_sep(x)
        x = self.conv_6_dw(x)
        x = self.flatten(x)
        x = self.linear(x)
        x = self.bn(x)
        x = self.dropout(x)
        x = self.prob(x)
        return x
'''

with open("minifasnet.py", "w") as f:
    f.write(minifasnet_full_code)

print("✅ minifasnet.py with only MiniFASNetV2 saved.")


✅ minifasnet.py with only MiniFASNetV2 saved.


In [75]:
import os, time, pickle
import cv2, face_recognition, numpy as np
import torch, torch.nn.functional as F
import torchvision.transforms as transforms
from minifasnet import MiniFASNetV2

# --- CONFIG ---
MODEL_FILENAME = "2.7_80x80_MiniFASNetV2.pth"
MODEL_PATH = os.path.join(os.getcwd(), MODEL_FILENAME)
ANTI_SPOOF_THRESHOLD = 0.5
FACE_MATCH_THRESHOLD = 0.4
DB_DIR = "db"

# --- Load model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

def load_anti_spoof_model(path):
    model = MiniFASNetV2(num_classes=3)
    sd = torch.load(path, map_location=device)
    if list(sd.keys())[0].startswith("module."):
        sd = {k.replace("module.", ""): v for k, v in sd.items()}
    model.load_state_dict(sd)
    return model.to(device).eval()

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model file not found at {MODEL_PATH}")
anti_spoof = load_anti_spoof_model(MODEL_PATH)

# --- Preprocess and inference ---
transform_as = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

def preprocess_face(face):
    face = cv2.cvtColor(cv2.resize(face, (80, 80)), cv2.COLOR_BGR2RGB)
    return transform_as(face).unsqueeze(0).to(device)

def predict_spoof(face_tensor):
    with torch.no_grad():
        logits = anti_spoof(face_tensor)
        prob = F.softmax(logits, dim=1)[0,0].item()
        label = "REAL" if prob > ANTI_SPOOF_THRESHOLD else "SPOOF"
        return label, prob

# --- Load face encodings ---
def load_encodings(db_dir):
    encs = {}
    for fname in os.listdir(db_dir):
        if fname.endswith(".pickle"):
            name = fname.split(".")[0].rsplit("_",1)[0]
            enc = pickle.load(open(os.path.join(db_dir, fname),"rb"))
            encs.setdefault(name, []).append(enc)
    return encs

encodeDict = load_encodings(DB_DIR)

# --- Main loop ---
def run():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open camera")
        return

    prev = time.time()
    print("Press Q to quit")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        small = cv2.resize(frame, (0,0), fx=0.25, fy=0.25)
        rgb = cv2.cvtColor(small, cv2.COLOR_BGR2RGB)
        locs = face_recognition.face_locations(rgb)
        feats = face_recognition.face_encodings(rgb, locs)

        for (y1,x2,y2,x1), enc in zip(locs, feats):
            y1,x1,y2,x2 = [v*4 for v in (y1,x1,y2,x1)]  # scale up coords
            face = frame[y1:y2, x1:x2]

            label, prob = predict_spoof(preprocess_face(face))

            if label == "REAL":
                name = "Unknown"
                dists = [(face_recognition.face_distance(enc_list, enc).min(), nm)
                         for nm, enc_list in encodeDict.items()]
                if dists and min(dists)[0] <= FACE_MATCH_THRESHOLD:
                    name = min(dists)[1].lower()
                color = (0,255,0) if name!="Unknown" else (255,255,0)
            else:
                name = "SPOOF DETECTED"
                color = (0,0,255)

            cv2.rectangle(frame, (x1,y1),(x2,y2),color,2)
            cv2.putText(frame, f"{label} {prob:.2f}", (x1,y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
            cv2.putText(frame, name, (x1,y2+20),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        fps = int(1/(time.time() - prev))
        prev = time.time()
        cv2.putText(frame, f"FPS: {fps}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        cv2.imshow("Anti‑Spoof + FaceRec", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    run()


Using device: cpu


RuntimeError: Error(s) in loading state_dict for MiniFASNetV2:
	Missing key(s) in state_dict: "conv2_dw.conv_dw.weight", "conv2_dw.bn_dw.weight", "conv2_dw.bn_dw.bias", "conv2_dw.bn_dw.running_mean", "conv2_dw.bn_dw.running_var", "conv2_dw.prelu_dw.weight", "conv_23.conv.weight", "conv_23.bn.weight", "conv_23.bn.bias", "conv_23.bn.running_mean", "conv_23.bn.running_var", "conv_23.prelu.weight", "conv_23.conv_dw.weight", "conv_23.bn_dw.weight", "conv_23.bn_dw.bias", "conv_23.bn_dw.running_mean", "conv_23.bn_dw.running_var", "conv_23.prelu_dw.weight", "conv_3.conv.weight", "conv_3.bn.weight", "conv_3.bn.bias", "conv_3.bn.running_mean", "conv_3.bn.running_var", "conv_3.prelu.weight", "conv_3.conv_dw.weight", "conv_3.bn_dw.weight", "conv_3.bn_dw.bias", "conv_3.bn_dw.running_mean", "conv_3.bn_dw.running_var", "conv_3.prelu_dw.weight", "conv_34.conv.weight", "conv_34.bn.weight", "conv_34.bn.bias", "conv_34.bn.running_mean", "conv_34.bn.running_var", "conv_34.prelu.weight", "conv_34.conv_dw.w